## 모듈 import

In [1]:
#모듈 import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from pycaret.regression  import *


# feature selection
from sklearn.linear_model import LinearRegression
import sys, warnings
if not sys.warnoptions: warnings.simplefilter("ignore")
from sklearn.feature_selection import SelectPercentile
from sklearn.model_selection import cross_val_score
from tqdm import tqdm

#Scailing
from sklearn.preprocessing import PowerTransformer

#Label Encoding
from sklearn.preprocessing import LabelEncoder

In [2]:
#삡운드
import winsound as sd
# 삡싸운드
def beepsound():
    fr = 2000   # range : 37 ~ 32767
    du = 1000 # 1000 ms ==1second
    sd.Beep(fr, du) # winsound.Beep(frequency, duration)

## data 불러오기

In [3]:
# 데이터가 저장된 경로 설정
data_path = '../data/'

# 데이터 불러오기
train = pd.read_csv(data_path + 'train_df.csv', encoding='cp949')
test = pd.read_csv(data_path + 'test_df.csv', encoding='cp949')
submission = pd.read_csv(data_path + 'sample_submission.csv')

In [4]:
train

,index,SEND_SPG_INNB,REC_SPG_INNB,DL_GD_LCLS_NM,DL_GD_MCLS_NM,INVC_CONT
0,0,1129000014045300,5011000220046300,패션의류,상의,3
1,1,1135000009051200,5011000178037300,생활/건강,반려동물,3
2,2,1135000030093100,5011000265091400,패션의류,기타패션의류,9
3,3,1154500002014200,5011000315087400,식품,농산물,10
4,4,1165000021008300,5011000177051200,식품,가공식품,3
...,...,...,...,...,...,...
31995,31995,5011001060063300,2635000026053400,식품,농산물,6
31996,31996,5011001095042400,1168000017002200,식품,기타식품,5
31997,31997,5011001108036200,4119700008012100,식품,농산물,9
31998,31998,5011001115011400,1132000015085100,식품,농산물,3


In [31]:
test

,index,SEND_SPG_INNB,REC_SPG_INNB,DL_GD_LCLS_NM,DL_GD_MCLS_NM
0,32000,5013000043028400,1165000021097200,식품,농산물
1,32001,5013000044016100,1154500002066400,식품,농산물
2,32002,5013000205030200,4139000102013200,식품,농산물
3,32003,5013000205030200,4221000040093400,식품,농산물
4,32004,5013000268011400,2726000004017100,식품,농산물
...,...,...,...,...,...
4635,36635,5013000858004400,4725000719072200,식품,농산물
4636,36636,5013000870018300,2826000106075300,식품,농산물
4637,36637,5013000897086300,4311100034004300,식품,농산물
4638,36638,5013000902065100,4145000013011200,식품,농산물


### 전처리

In [5]:
data = pd.concat([train,test]).reset_index(drop=True);data

,index,SEND_SPG_INNB,REC_SPG_INNB,DL_GD_LCLS_NM,DL_GD_MCLS_NM,INVC_CONT
0,0,1129000014045300,5011000220046300,패션의류,상의,3.0
1,1,1135000009051200,5011000178037300,생활/건강,반려동물,3.0
2,2,1135000030093100,5011000265091400,패션의류,기타패션의류,9.0
3,3,1154500002014200,5011000315087400,식품,농산물,10.0
4,4,1165000021008300,5011000177051200,식품,가공식품,3.0
...,...,...,...,...,...,...
36635,36635,5013000858004400,4725000719072200,식품,농산물,NaN
36636,36636,5013000870018300,2826000106075300,식품,농산물,NaN
36637,36637,5013000897086300,4311100034004300,식품,농산물,NaN
36638,36638,5013000902065100,4145000013011200,식품,농산물,NaN


- Make Feature

In [6]:
# 송하인 기준으로 총 몇개의 택배를 보냈는지 확인하여 이를 피처로 만들어줌 
groupby_SEND_SPG_INNB_sum = data.groupby('SEND_SPG_INNB').sum()['INVC_CONT'].reset_index()
# 이후에 피처로 만들어주기 위한 dic생성
groupby_SEND_SPG_INNB_sum_dix = {x:y for x,y in zip(groupby_SEND_SPG_INNB_sum['SEND_SPG_INNB'], groupby_SEND_SPG_INNB_sum['INVC_CONT'])}
# 피처 생성
data['총물류량'] = data['SEND_SPG_INNB'].apply(lambda x: groupby_SEND_SPG_INNB_sum_dix[x])

In [7]:
# 송하인 기준으로 총 몇명의 수하인에게 택배를 보냈는지 확인하여 이를 피처로 만들어줌 
groupby_SEND_SPG_INNB_count = data.groupby('SEND_SPG_INNB').count()['REC_SPG_INNB'].reset_index()
# 이후에 피처로 만들어주기 위한 dic생성
groupby_SEND_SPG_INNB_count_dix = {x:y for x,y in zip(groupby_SEND_SPG_INNB_count['SEND_SPG_INNB'], groupby_SEND_SPG_INNB_count['REC_SPG_INNB'])}
# 피처 생성
data['거래수하인수'] = data['SEND_SPG_INNB'].apply(lambda x: groupby_SEND_SPG_INNB_count_dix[x])

In [8]:
# 평균 한건당 몇번의 수화물을 보내는지에 대한 피처 생성
data['평균거래량'] = data['총물류량'] / data['거래수하인수']

In [9]:
# dic_max = {x: y for x, y in zip(data.groupby('SEND_SPG_INNB').max()[['DL_GD_LCLS_NM', '거래수하인수']].query('거래수하인수 > 100').reset_index()['SEND_SPG_INNB'], 
#                      data.groupby('SEND_SPG_INNB').max()[['DL_GD_LCLS_NM', '거래수하인수']].query('거래수하인수 > 100').reset_index()['DL_GD_LCLS_NM'])}
# data['최대거래품목'] = data['SEND_SPG_INNB'].apply(lambda x: dic_max[x] if x in dic_max else '거래량적음')

In [10]:
# 인코딩과 스케일링을 위함
data_index = data['index']
y_data = data['INVC_CONT']

data = data.drop(columns=['index', 'INVC_CONT'])

- 결측치 채움 -> 없음

- Encoding, Scailing

In [11]:
# # 수동으로 범주형 변수로 만들어줌
# data['SEND_SPG_INNB'] = data['SEND_SPG_INNB'].astype('str')
# data['REC_SPG_INNB'] = data['REC_SPG_INNB'].astype('str')

In [12]:
# 범주형 변수와 수치형 변수를 분리
cat_features = data.select_dtypes(include=['object']).columns.to_list()
num_features = data.select_dtypes(exclude='object').columns.to_list() 

In [13]:
# scailing을 위해
num_features = num_features[2:]

In [14]:
#수치형변수 scailing
data[num_features] = PowerTransformer(standardize=True).fit_transform(data[num_features])

# # #라벨인코딩
# # #위의 범주형 변수들을 라벨인코딩
# for f in cat_features:
#     data[f] = LabelEncoder().fit_transform(data[f])

# # 원핫인코딩
# data = pd.get_dummies(data)

In [15]:
#위에서 떼어놨던 피처 병합
data = pd.concat([data_index, data],axis=1)
data = pd.concat([data, y_data],axis=1)

### W2V

In [16]:
# oversampling
import random
# w2v
from gensim.models import word2vec

In [17]:
# 오버샘플링
def oversample(x, n):
    lst = []
    for i in x:
        tmp = []
        for j in range(n):
            random.shuffle(i)
            tmp += list(i)
        lst.append(tmp)
    return lst

In [18]:
cat_features_w2v = []
# 모든 범주형 피처를 w2v
for level in tqdm(cat_features):
    # W2V 학습을 하기에는 데이터(즉 corpus)가 부족하여 oversampling을 수행
    sentences = [ list(data.query('index == @id')[level].values) for id in data['index'].unique() ]

    # oversampling
    w2v_input = oversample(sentences, 5)

    max_features = 300 # 문자 벡터 차원 수
    min_word_count = 1 # 최소 문자 수
    num_workers = 4 # 병렬 처리 스레드 수
    context = 3 # 문자열 창 크기
    downsampling = 1e-3 # 문자 빈도수 Downsample

    # 모델 학습
    w2v = word2vec.Word2Vec(sentences = w2v_input,
                            vector_size = 30,
                            window = 3,
                            min_count = 1,
                            sg = 1).wv
    # 필요없는 메모리 unload
    w2v.init_sims(replace=True)

    #train
    train_mean_vector = []

    for words in sentences:
            tmp = np.zeros(30)             # 다음 customer ID에 대한 vector를 계산하기 전 0으로 초기화
            cnt = 0
            for word in words:
                try:
                    tmp += w2v[word]
                    cnt += 1
                except:
                    pass
            tmp /= cnt                      # customer ID 에 있는 아이템 갯수로 전체 벡터를 mean해줌  
            train_mean_vector.append(tmp)

    train_mean_vector = np.array(train_mean_vector)

    locals()[f'data_{level}'] = pd.DataFrame(train_mean_vector)
    cat_features_w2v.append(locals()[f'data_{level}'])

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [01:50<00:00, 55.17s/it]


In [19]:
w2v_data = pd.concat(cat_features_w2v,axis=1)

In [20]:
data = pd.concat([data_index,data[num_features], w2v_data, y_data],axis=1);data.head(3)

,index,총물류량,거래수하인수,평균거래량,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,INVC_CONT
0,0,276.0,75,3.680000,0.078010,-0.062008,0.202466,0.007812,0.278174,0.177194,-0.210888,-0.135668,-0.01536,0.246952,-0.260440,0.134137,0.092477,0.156668,0.258267,-0.176697,0.112003,0.229039,-0.056133,-0.175378,0.023042,-0.305196,-0.160612,0.283288,0.148857,0.209061,0.229684,0.241352,-0.153014,-0.019378,-0.037612,-0.215321,0.183008,0.188093,-0.062219,-0.064370,-0.007769,0.051458,-0.301358,-0.148084,0.335130,-0.004399,0.070634,-0.254125,0.119395,-0.147678,0.011620,0.015759,-0.231974,0.080955,0.067941,-0.05030,-0.294272,0.262465,0.260516,-0.106996,0.149692,0.162888,0.176270,-0.399013,3.0
1,1,133.0,33,4.030303,-0.173027,-0.092813,0.373588,-0.166710,-0.085782,-0.028706,0.280428,-0.067925,-0.06051,-0.063889,-0.274318,0.191677,-0.307152,-0.106479,0.048420,-0.026413,-0.104089,0.323790,0.114620,0.183694,-0.175278,0.065134,-0.076450,0.139225,0.308340,-0.126828,0.170170,-0.131824,-0.181791,0.246786,0.022068,0.187093,0.260544,-0.074944,0.205772,-0.136136,-0.101209,0.273203,-0.080043,-0.290480,0.261134,-0.130784,0.008586,0.268556,-0.093196,-0.113228,-0.115095,0.234254,-0.290276,0.086751,-0.095676,0.24291,0.063370,0.109871,-0.017212,-0.046741,0.260812,0.175358,-0.074930,-0.352767,3.0
2,2,26.0,2,13.000000,0.078010,-0.062008,0.202466,0.007812,0.278174,0.177194,-0.210888,-0.135668,-0.01536,0.246952,-0.260440,0.134137,0.092477,0.156668,0.258267,-0.176697,0.112003,0.229039,-0.056133,-0.175378,0.023042,-0.305196,-0.160612,0.283288,0.148857,0.209061,0.229684,0.241352,-0.153014,-0.019378,0.213685,0.115256,0.269326,-0.104784,-0.144897,-0.273202,-0.297042,-0.267398,0.006662,-0.179797,0.160392,-0.160784,0.028937,0.047752,-0.137183,0.030172,0.365333,0.259554,-0.095886,0.159471,-0.183026,0.15214,0.043878,0.139007,0.073556,0.182912,-0.141669,0.238218,0.195369,-0.167820,9.0


In [21]:
# 열 중복 해결
col_name = ['index', 'SEND_SPG_INNB', 'REC_SPG_INNB']
for i in range(len(data.columns)-len(col_name)-1):
    col_name.append(i)
col_name.append('INVC_CONT')
data.columns = col_name

In [22]:
data

,index,SEND_SPG_INNB,REC_SPG_INNB,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,INVC_CONT
0,0,276.0,75,3.680000,0.078010,-0.062008,0.202466,0.007812,0.278174,0.177194,-0.210888,-0.135668,-0.01536,0.246952,-0.260440,0.134137,0.092477,0.156668,0.258267,-0.176697,0.112003,0.229039,-0.056133,-0.175378,0.023042,-0.305196,-0.160612,0.283288,0.148857,0.209061,0.229684,0.241352,-0.153014,-0.019378,-0.037612,-0.215321,0.183008,0.188093,-0.062219,-0.064370,-0.007769,0.051458,-0.301358,-0.148084,0.335130,-0.004399,0.070634,-0.254125,0.119395,-0.147678,0.011620,0.015759,-0.231974,0.080955,0.067941,-0.050300,-0.294272,0.262465,0.260516,-0.106996,0.149692,0.162888,0.176270,-0.399013,3.0
1,1,133.0,33,4.030303,-0.173027,-0.092813,0.373588,-0.166710,-0.085782,-0.028706,0.280428,-0.067925,-0.06051,-0.063889,-0.274318,0.191677,-0.307152,-0.106479,0.048420,-0.026413,-0.104089,0.323790,0.114620,0.183694,-0.175278,0.065134,-0.076450,0.139225,0.308340,-0.126828,0.170170,-0.131824,-0.181791,0.246786,0.022068,0.187093,0.260544,-0.074944,0.205772,-0.136136,-0.101209,0.273203,-0.080043,-0.290480,0.261134,-0.130784,0.008586,0.268556,-0.093196,-0.113228,-0.115095,0.234254,-0.290276,0.086751,-0.095676,0.242910,0.063370,0.109871,-0.017212,-0.046741,0.260812,0.175358,-0.074930,-0.352767,3.0
2,2,26.0,2,13.000000,0.078010,-0.062008,0.202466,0.007812,0.278174,0.177194,-0.210888,-0.135668,-0.01536,0.246952,-0.260440,0.134137,0.092477,0.156668,0.258267,-0.176697,0.112003,0.229039,-0.056133,-0.175378,0.023042,-0.305196,-0.160612,0.283288,0.148857,0.209061,0.229684,0.241352,-0.153014,-0.019378,0.213685,0.115256,0.269326,-0.104784,-0.144897,-0.273202,-0.297042,-0.267398,0.006662,-0.179797,0.160392,-0.160784,0.028937,0.047752,-0.137183,0.030172,0.365333,0.259554,-0.095886,0.159471,-0.183026,0.152140,0.043878,0.139007,0.073556,0.182912,-0.141669,0.238218,0.195369,-0.167820,9.0
3,3,10.0,1,10.000000,0.191515,-0.204376,0.286949,0.291666,-0.139717,-0.038913,0.206452,0.384369,-0.16121,0.126825,0.187791,-0.140552,-0.149376,0.167083,-0.046367,0.037925,0.276053,0.020851,-0.010166,-0.037746,0.318667,-0.160964,0.105655,0.082364,0.170346,-0.065239,-0.101188,0.220109,-0.101703,-0.256821,-0.139334,0.089414,-0.014557,0.286161,-0.199861,-0.236544,0.154141,0.062687,-0.057748,-0.143198,0.299620,-0.163823,0.029203,0.188569,-0.174722,0.026357,0.182072,-0.082588,-0.228577,0.087788,0.294514,0.216770,0.048226,-0.143136,0.001862,-0.015508,-0.206156,0.311626,0.083984,-0.394267,10.0
4,4,3.0,1,3.000000,0.191515,-0.204376,0.286949,0.291666,-0.139717,-0.038913,0.206452,0.384369,-0.16121,0.126825,0.187791,-0.140552,-0.149376,0.167083,-0.046367,0.037925,0.276053,0.020851,-0.010166,-0.037746,0.318667,-0.160964,0.105655,0.082364,0.170346,-0.065239,-0.101188,0.220109,-0.101703,-0.256821,0.068475,-0.077172,0.294029,-0.308143,0.262555,0.027697,-0.117748,0.113906,-0.046247,0.053687,0.418332,-0.251058,-0.069678,-0.100878,0.042123,0.128055,0.263805,0.180918,0.169731,0.307694,0.190095,-0.063834,-0.198455,-0.001983,-0.070663,0.269488,0.188480,0.077216,0.037877,-0.095943,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36635,36635,215.0,56,3.839286,0.191515,-0.204376,0.286949,0.291666,-0.139717,-0.038913,0.206452,0.384369,-0.16121,0.126825,0.187791,-0.140552,-0.149376,0.167083,-0.046367,0.037925,0.276053,0.020851,-0.010166,-0.037746,0.318667,-0.160964,0.105655,0.082364,0.170346,-0.065239,-0.101188,0.220109,-0.101703,-0.256821,-0.139334,0.089414,-0.014557,0.286161,-0.199861,-0.236544,0.154141,0.062687,-0.057748,-0.143198,0.299620,-0.163823,0.029203,0.188569,-0.174722,0.026357,0.182072,-0.082588,-0.228577,0.087788,0.294514,0.216770,0.048226,-0.143136,0.001862,-0.015508,-

- Data split

In [23]:
#train과 test를 분리
train = data.iloc[:32000,:]
test = data.iloc[32000:,:]
test = test.drop(columns = ['INVC_CONT']).reset_index(drop=True)

In [24]:
train

,index,SEND_SPG_INNB,REC_SPG_INNB,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,INVC_CONT
0,0,0.075486,-0.055179,0.194575,-0.002269,0.285731,0.176562,-0.225883,-0.140628,-0.012406,0.246275,-0.248632,0.130252,0.101921,0.153156,0.259022,-0.177229,0.110342,0.228211,-0.056029,-0.176404,0.026345,-0.302891,-0.167539,0.276827,0.137936,0.219579,0.235069,0.238993,-0.149408,-0.015764,-0.036601,-0.217248,0.182600,0.186319,-0.061225,-0.065311,-0.005829,0.051549,-0.302444,-0.148690,0.333724,-0.003769,0.069287,-0.252282,0.120193,-0.146157,0.013847,0.013925,-0.231487,0.081269,0.066066,-0.052618,-0.292867,0.263809,0.259501,-0.107480,0.150989,0.165559,0.177202,-0.399877,3.0
1,1,-0.172406,-0.093881,0.385244,-0.174371,-0.078323,-0.028321,0.269469,-0.062208,-0.065662,-0.056325,-0.260033,0.190873,-0.305419,-0.105799,0.055034,-0.030552,-0.093529,0.335567,0.113119,0.176337,-0.161609,0.059169,-0.084496,0.144996,0.308873,-0.112793,0.184694,-0.125151,-0.190378,0.249170,0.021204,0.185887,0.260918,-0.075620,0.206877,-0.135647,-0.101720,0.273522,-0.081498,-0.289923,0.259999,-0.130045,0.009009,0.267088,-0.092609,-0.113470,-0.115088,0.236213,-0.291526,0.086776,-0.094668,0.243306,0.060150,0.109569,-0.017129,-0.047449,0.260831,0.174500,-0.075465,-0.353126,3.0
2,2,0.075486,-0.055179,0.194575,-0.002269,0.285731,0.176562,-0.225883,-0.140628,-0.012406,0.246275,-0.248632,0.130252,0.101921,0.153156,0.259022,-0.177229,0.110342,0.228211,-0.056029,-0.176404,0.026345,-0.302891,-0.167539,0.276827,0.137936,0.219579,0.235069,0.238993,-0.149408,-0.015764,0.212939,0.115125,0.271438,-0.104979,-0.142768,-0.270536,-0.295203,-0.268753,0.006136,-0.178725,0.161661,-0.160861,0.027672,0.044100,-0.134690,0.030678,0.367000,0.260404,-0.096554,0.159907,-0.182778,0.152348,0.042273,0.140056,0.074437,0.184263,-0.141193,0.239201,0.194881,-0.167147,9.0
3,3,0.190087,-0.199767,0.294540,0.279262,-0.126242,-0.040868,0.184374,0.383624,-0.162994,0.132053,0.209291,-0.147912,-0.139948,0.169141,-0.041467,0.032850,0.281577,0.032903,-0.010954,-0.042537,0.335207,-0.165288,0.089933,0.079832,0.165202,-0.043864,-0.084337,0.226547,-0.106885,-0.253482,-0.135650,0.087745,-0.019441,0.282935,-0.200153,-0.233173,0.153410,0.064352,-0.055416,-0.140224,0.299443,-0.165554,0.033386,0.190968,-0.178985,0.027509,0.182891,-0.087654,-0.226721,0.087626,0.298190,0.216221,0.051826,-0.147562,0.004747,-0.012550,-0.204574,0.308720,0.082364,-0.395871,10.0
4,4,0.190087,-0.199767,0.294540,0.279262,-0.126242,-0.040868,0.184374,0.383624,-0.162994,0.132053,0.209291,-0.147912,-0.139948,0.169141,-0.041467,0.032850,0.281577,0.032903,-0.010954,-0.042537,0.335207,-0.165288,0.089933,0.079832,0.165202,-0.043864,-0.084337,0.226547,-0.106885,-0.253482,0.069172,-0.078548,0.291904,-0.310891,0.262703,0.025334,-0.118215,0.114521,-0.045149,0.052253,0.416289,-0.251998,-0.069136,-0.097923,0.040759,0.128852,0.266557,0.177383,0.170787,0.308062,0.189190,-0.066532,-0.195205,-0.001993,-0.071571,0.270880,0.189700,0.078141,0.037595,-0.096530,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31995,31995,0.190087,-0.199767,0.294540,0.279262,-0.126242,-0.040868,0.184374,0.383624,-0.162994,0.132053,0.209291,-0.147912,-0.139948,0.169141,-0.041467,0.032850,0.281577,0.032903,-0.010954,-0.042537,0.335207,-0.165288,0.089933,0.079832,0.165202,-0.043864,-0.084337,0.226547,-0.106885,-0.253482,-0.135650,0.087745,-0.019441,0.282935,-0.200153,-0.233173,0.153410,0.064352,-0.055416,-0.140224,0.299443,-0.165554,0.033386,0.190968,-0.178985,0.027509,0.182891,-0.087654,-0.226721,0.087626,0.298190,0.216221,0.051826,-0.147562,0.004747,-0.012550,-0.204574,0.308720,0.082364,-0.395871,6.0
31996,31996,0.190087,-0.199767,0.294540,0.279262,-0.126242,-0.040868,0.1843

## Modeling with pycaret

- Model setting

In [25]:
# #pycaret에 넣도록 변형
# train = pd.concat([pd.DataFrame(train),y_train],axis=1)
# test = pd.concat([test_index, pd.DataFrame(test)],axis=1)

In [26]:
train = train.iloc[:,1:]

In [27]:
reg = setup(train, 
            preprocess = False, # True로 설정되면, 자체적인 Feature Engineering을 추가로 진행해 Predict가 불가능해진다.
            train_size = 0.999,  # 우리는 전체 데이터를 학습해 test를 예측하는게 목표이기 때문에, 0.999로 설정한다.
            target = 'INVC_CONT', # 목표 변수는 매칭성공여부이다.
            silent = True, # 엔터를 누르기 귀찮다. 궁금하면 풀어보세요
            use_gpu = True, # GPU가 있으면 사용하세요 (Cat BOost 속도 향상)
#             numeric_features=['전체회차','현재회차'], # 숫자로써의 의미가 있다.
            session_id = 42,
            fold_shuffle = True
            )

,Description,Value
0,session_id,42
1,Target,INVC_CONT
2,Original Data,"(32000, 61)"
3,Missing Values,False
4,Numeric Features,60
5,Categorical Features,0
6,Transformed Train Set,"(31967, 60)"
7,Transformed Test Set,"(33, 60)"
8,Shuffle Train-Test,True
9,Stratify Train-Test,False


In [28]:
top5 = compare_models(sort='RMSE', fold=5, n_select = 5)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
br,Bayesian Ridge,2.2114,32.1261,5.6641,0.0294,0.4164,0.4276,0.0660
ridge,Ridge Regression,2.2114,32.1277,5.6643,0.0293,0.4166,0.4276,0.0180
xgboost,Extreme Gradient Boosting,2.2114,32.1286,5.6644,0.0293,0.4166,0.4276,0.3600
gbr,Gradient Boosting Regressor,2.2114,32.1286,5.6644,0.0293,0.4166,0.4276,2.6100
catboost,CatBoost Regressor,2.2114,32.1286,5.6644,0.0293,0.4166,0.4276,3.7200
dt,Decision Tree Regressor,2.2114,32.1286,5.6644,0.0293,0.4166,0.4276,0.0540
et,Extra Trees Regressor,2.2114,32.1286,5.6644,0.0293,0.4166,0.4276,0.7540
lightgbm,Light Gradient Boosting Machine,2.2114,32.1286,5.6644,0.0293,0.4166,0.4276,0.6060
rf,Random Forest Regressor,2.2127,32.1296,5.6645,0.0293,0.4168,0.4280,0.8060
omp,Orthogonal Matching Pursuit,2.2247,32.2660,5.6763,0.0253,0.4185,0.4309,0.0200


- Hyperparameter tuning

In [29]:
#catboost 모델 생성
catboost = create_model('catboost')
catboost = tune_model(catboost, optimize = 'RMSE', choose_better = True, fold = 5, n_iter = 100)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,2.1897,29.5894,5.4396,0.0163,0.4146,0.4295
1,2.1683,30.3271,5.5070,0.0288,0.4116,0.4283
2,2.2125,33.9185,5.8240,0.0374,0.4146,0.4190
3,2.2426,35.9134,5.9928,0.0291,0.4200,0.4283
4,2.2453,30.8778,5.5568,0.0355,0.4184,0.4340
Mean,2.2117,32.1253,5.6640,0.0294,0.4158,0.4278
SD,0.0298,2.3994,0.2098,0.0074,0.0030,0.0049


In [28]:
# top5 = [top5[0], top5[1], top5[3],top5[4]]
# top5.append(catboost)

In [36]:
# # hyperparameter tuning
# models = []
# for m in top5:
#     if top5.index(m) == 2: #오래걸리는 모델은 뺌
#         models.append(tune_model(m, optimize = 'RMSE', choose_better = True, fold = 5, n_iter = 100))
#     else:
#         models.append(m)

### Ensemble

- blend model

In [35]:
# final_model = blend_models(estimator_list = models, optimize = 'RMSE')
# # final_model = finalize_model(final_model) # 전체 데이터로 재학습

- Fit & predict

In [30]:
model = catboost
model.fit(train.iloc[:,:-1],train['INVC_CONT'])
pred = model.predict(test.iloc[:,1:])

In [31]:
submission['INVC_CONT'] = pred

### Make submission 

In [32]:
t = pd.Timestamp.now()
fname = f'submission_{t.month:02}{t.day:02}{t.hour:02}{t.minute:02}.csv'
# fname = 'submission_etpercentile.csv'
submission.to_csv('../submission/'+fname, index=False)
print("'{}' is ready to submit." .format(fname))

'submission_12211443.csv' is ready to submit.


In [33]:
beepsound()

# END